In [1]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

In [2]:
import os
os.chdir('/Users/asudomoeva/Desktop/Capstone')

# Working with industry data

In [25]:
industry_target = pd.read_csv('Data and Preprocessing/Data/industry_target from sba.csv')
industry_data = pd.read_csv('Data and Preprocessing/Data/industry_allyear.csv')
industry_index = pd.read_csv('Data and Preprocessing/Data/Codes.csv')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (10,19,20,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Let's start with mapping targets from SBA

In [26]:
# reading in the index file
industry_index = industry_index.rename(index=str, columns={"2017 NAICS US   Code": "NAICS"})
industry_index['NAICS'] = pd.to_numeric(industry_index['NAICS'])

In [27]:
industry_target=industry_target.merge(industry_index, on='NAICS', how='left')

In [28]:
# filtering for those firms of size less than 500 employees 
ind_state_target = industry_target.loc[industry_target['firm type'].isin(['firm', 'firms'])][['Year', 
                                                        'State', 'NAICS', '<500', '2017 NAICS US Title']]
ind_state_target['<500'] = pd.to_numeric(ind_state_target['<500'])
ind_state_target['Year'] = ind_state_target['Year'].astype(int)

In [29]:
# create one for the US
ind_us_target = ind_state_target.groupby(['NAICS', 'Year']).sum()

In [30]:
ind_state_target.head()

,Year,State,NAICS,<500,2017 NAICS US Title
0,2011,Alabama,113,586.0,Forestry and Logging
1,2011,Alabama,114,22.0,"Fishing, Hunting and Trapping"
2,2011,Alabama,115,163.0,Support Activities for Agriculture and Forestry
3,2011,Alabama,211,18.0,Oil and Gas Extraction
4,2011,Alabama,212,78.0,Mining (except Oil and Gas)


In [31]:
ind_us_target.head()

<500
NAICS Year         
113   1998  13982.0
      1999  13444.0
      2000  13099.0
      2001  12561.0
      2002  12274.0

## Now we can take a look at the data from CENSUS

The feature description table:

|  Name | Type   |  Description |
|---|---|---|
|  STATE | 	C   | Geographic Area Code, FIPS State 2-digit codes  |
| NAICS  |  C   |  Industry Code; 6-digit, North American Industry Classification System (NAICS) |
|  ENTRSIZE |  C  |  Enterprise Employment Size Code |
|  INIT_ESTB |  N  |  Number of Initial Year Establishments |
|  INIT_EMPL |  N  | Initial Year Employment with Noise  |
|  INIT_EMPLFL_N |  C  | Initial Year Employment Noise Flag   |
| INIT_EMPFL_R  | C   |   Initial Year Employment Range Flag / Data Suppression Flag |
|  NETCHG_ESTB |  N |  Change in Establishments |
| NETCHG_EMPL   |  N |  Change in Employment |
|  NETCHG_EMPLFL_N  | C  | Net Change Noise Flag (G,H,D,S... see INIT_EMPLFL_N)  |
|  BIRTHS_ESTB  | N   |  Number of Establishment Births  |
|  BIRTHS_EMPL  |  N  |  Establishment Births Employment with Noise  |
|   BIRTHS_EMPLFL_N|  C   |  Establishment Births Noise Flag (G,H,D,S... see INIT_EMPLFL_N  |
|  DEATHS_ESTB  |  N   |  Number of Establishment Deaths  |
|  DEATHS_EMPL   |   N  |  Establishment Deaths Employment with Noise  |
|  DEATHS_EMPLFL_N  |  C   |  Establishment Deaths Noise Flag (G,H,D,S... see INIT_EMPLFL_N)  |
|   EXP_ESTB |  N   |  Number of Establishment Expansions  |
|   EXP_EMPL |  N   |   Establishment Expansions Change in Employment with Noise |
|  EXP_EMPLFL_N  |  C   |  Establishment Expansion Noise Flag  |
|  CONTR_ESTB  |   N  |  Number of Establishment Contractions  |
|  CONTR_EMPL  |   N |  Establishment Contractions Change in Employment with Noise  |
|  CONTR_EMPLFL_N  |   C  | Establish Contractors Noise Flag   |
|   PCTCHG_ESTB |  N   |  Percent Change Establishments  |
|  PCTCHG_EMPL  |  N  |   Percent Change in Employment |
|  PCTCHG_BIRTHS_EMPL  |  N   |  Percent Change in Employment Due to Births  |
|  PCTCHG_DEATHS_EMPL  |  N   |   Percent Change in Employment Due to Deaths |
|  PCTCHG_BIRTHS_EXP_EMPL  |  N   |   Percent Change in Employment Due to Births & Expansion |
|  PCTCGH_DEATHS_CONTR_EMPL  | N   | Percent change in Employment Due to Deaths & Contractions    |
|  STATEDSCR  |  C   |  State Description  |
|  NAICSDSCR  |   C  |  NAICS Industry Description  |
|  ENTRSIZEDSCR  |  C   |  Enterprise Employment Size Description  |

This denotes employment size class for data withheld to avoid disclosure (confidentiality) or withheld because data do not meet publication standards.
 
1. A:        0-19
2. B:        20-99
3. C:        100-249
4. E:        250-499
5. F:        500-999
6. G:        1,000-2,499
7. H:        2,500-4,999
8. I:        5,000-9,999
9. J:        10,000-24,999
10. K:        25,000-49,999
11. L:        50,000-99,999
12. M:        100,000 or More

_____________________
Other marks
1. G:        Low noise applied to cell value (0 to < 2%)
2. H:        Medium noise applied to cell value (2 to < 5%)
3. D:        Data withheld and value set to 0 to avoid disclosing data for individual businesses; data are included in higher level totals. 
4. S:       Data withheld and value set to 0 to avoise releasing information that does not meet publication standards; data are included in higher level totals.

Will be treating as targets:
1. Number of Initial Year Establishments
2. Change in Establishments
3. Number of Establishment Births
4. Number of Establishment Deaths
5. Number of Establishment Expansions
6. Number of Establishment Contractions
7. Percent Change Establishments

In [32]:
# split the US data with States and using '7' to filter for industries with less than 500 employees
industry_us_data = industry_data.loc[(industry_data['STATE'] == 0) & (industry_data['ENTRSIZE'] == 7)][['NAICS',
                                                    'INIT_ESTB', 'NETCHG_ESTB', 'BIRTHS_ESTB', 'DEATHS_ESTB', 
                                                'EXP_ESTB', 'CONTR_ESTB', 'PCTCHG_ESTB','NCSDSCR','year' ]]
industry_state_data = industry_data.loc[(industry_data['STATE'] != 0) & (industry_data['ENTRSIZE'] == 7)][['STATE',
                                                 'NAICS', 'INIT_ESTB', 'NETCHG_ESTB', 'BIRTHS_ESTB', 'DEATHS_ESTB', 
                                             'EXP_ESTB', 'CONTR_ESTB', 'PCTCHG_ESTB','NCSDSCR','year', 'STATEDSCR']]

In [33]:
industry_us_data.head(3)

,NAICS,INIT_ESTB,NETCHG_ESTB,BIRTHS_ESTB,DEATHS_ESTB,EXP_ESTB,CONTR_ESTB,PCTCHG_ESTB,NCSDSCR,year
6,--,5775217,-47102,625873,672975,1462447,1505943,-0.8,NaN,2008
14,11,19350,-662,2096,2758,4230,4751,-3.4,NaN,2008
22,113,9183,-551,760,1311,2069,2617,-6.0,NaN,2008


In [34]:
industry_state_data.head(3)

,STATE,NAICS,INIT_ESTB,NETCHG_ESTB,BIRTHS_ESTB,DEATHS_ESTB,EXP_ESTB,CONTR_ESTB,PCTCHG_ESTB,NCSDSCR,year,STATEDSCR
3218,1,--,78993,-861,7720,8581,20604,21860,-1.1,NaN,2008,Alabama
3226,1,11,845,-34,55,89,198,265,-4.0,NaN,2008,Alabama
3234,1,21,157,-4,13,17,47,58,-2.5,NaN,2008,Alabama


_________

# Adding the industry targets and description into our final dataset

## First let's take a look at states full dataset

In [35]:
# create a 1:1 match between a state and a top industry to add into the bigger state regression
state_industry = pd.read_csv('Data and Preprocessing/Data/State to Industry Mapping.csv')
temp_data = pd.read_csv('Data and Preprocessing/Data/final_data(industry not added).csv')
temp_data = temp_data.replace({'(NA)': None, '(L)': 75000})
temp_data = temp_data.drop(['Compensation of employees', 'Gross operating surplus', 'Per capita real GDP by state',
                           'Subsidies', 'Taxes on production and imports', 
                            'Taxes on production and imports less subsidies', 'Forest and maple products'], axis=1)
final_data = temp_data.merge(state_industry, on='state', how='left')
final_data['GDP PC'] = temp_data['Real GDP by state']/temp_data['Population (persons) ']
final_data = final_data.dropna()
final_data['NAICS'] = final_data['NAICS'].astype(int)

In [36]:
# turning top industry into dummies
industry_dummies = pd.get_dummies(final_data['Top Industry'])
final_data = pd.concat([final_data, industry_dummies], axis=1, sort=False)

In [37]:
# outputting final data for states
final_data.to_csv('final_data_states.csv')

## Merging datatsets for indsutry target and states data

## Just industry data

In [ ]:
# need more industry indicators

In [ ]:
# output the final file

## Industry-State Data

In [38]:
final_data_sub = final_data[final_data.columns[:-16]]

In [39]:
final_data_sub['unique_name'] = final_data_sub['year'].astype(str) + '_' + final_data_sub[
    'GeoName']

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [40]:
industry_state_data['unique_name'] = industry_state_data['year'].astype(str) + '_' + industry_state_data[
    'STATEDSCR'].astype(str)
industry_state_data = industry_state_data.loc[industry_state_data['NAICS'] != '--']

In [41]:
industry_descriptions = industry_state_data['NCSDSCR']
industry_state_data = industry_state_data.drop(['NCSDSCR'], axis=1)
industry_state_data = industry_state_data.merge(final_data_sub, on='unique_name', how='left')

### Working with NAICS codes
1. 31-33 - turning into 31 for usability (will need to check for 32 and 33 when merging)
2. 44-45 - turning into 44
3. 48-49 - turning into 48

In [43]:
industry_state_data['NAICS_x'] = industry_state_data['NAICS_x'].replace({'31-33': 31, '44-45': 44, '48-49':48})

In [44]:
#trimming NAICS_y (top industry in the state)
industry_state_data['NAICS_y'] = industry_state_data['NAICS_y'].apply(lambda x: str(x)[:2])

In [47]:
industry_state_data = industry_state_data.replace({'na': None}).dropna()

In [48]:
industry_state_data['top_industry_bool'] = (pd.to_numeric(industry_state_data[
            'NAICS_x']) == pd.to_numeric(industry_state_data['NAICS_y'])).astype(int)

In [49]:
industry_state_data

,STATE,NAICS_x,INIT_ESTB,NETCHG_ESTB,BIRTHS_ESTB,DEATHS_ESTB,EXP_ESTB,CONTR_ESTB,PCTCHG_ESTB,year_x,...,contract_establish,end_establish,expand_establish,net_change,open_establish,total_contract,total_expand,Top Industry,NAICS_y,top_industry_bool
0,1,11,845,-34,55,89,198,265,-4.0,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
1,1,21,157,-4,13,17,47,58,-2.5,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
2,1,22,118,-2,1,3,31,22,-1.7,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
3,1,23,9151,-335,1032,1367,2229,2983,-3.7,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
4,1,31,3843,-10,279,289,1218,1379,-0.3,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
5,1,42,4356,-57,281,338,1198,1125,-1.3,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
6,1,44,12643,-347,1190,1537,3152,3587,-2.7,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
7,1,48,2398,-98,279,377,551,678,-4.1,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
8,1,51,754,-24,57,81,207,176,-3.2,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0
9,1,52,4024,4,327,323,931,788,0.1,2008,...,751202.0,128306.0,610200.0,-135240.0,134068.0,879508.0,744268.0,Ambulatory health care services,62,0


In [ ]:
# output the final file